In [5]:
import Agent
import pandas as pd
pd.set_option('display.max_columns', None)

chess_data_path = r'C:\Users\Abrah\Dropbox\PC (2)\Desktop\GitHub Repos\CST499-40_FA22-Capstone-BradleyChess\chess_data\chess_data_cleaned.pkl'
chess_data = pd.read_pickle(chess_data_path, compression = 'zip')

In [6]:
w_agent = Agent.Agent('W', chess_data)

In [27]:
w_agent.Q_table

,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,W11,W12,W13,W14,W15,W16,W17,W18,W19,W20,W21,W22,W23,W24,W25,W26,W27,W28,W29,W30,W31,W32,W33,W34,W35,W36,W37,W38,W39,W40,W41,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51,W52,W53,W54,W55,W56,W57,W58,W59,W60,W61,W62,W63,W64,W65,W66,W67,W68,W69,W70,W71,W72,W73,W74,W75,W76,W77,W78,W79,W80,W81,W82,W83,W84,W85,W86,W87,W88,W89,W90,W91,W92,W93,W94,W95,W96,W97,W98,W99,W100,W101,W102,W103,W104,W105,W106,W107,W108,W109,W110,W111,W112,W113,W114,W115,W116,W117,W118,W119,W120,W121,W122,W123,W124,W125,W126,W127,W128,W129,W130,W131,W132,W133,W134,W135,W136,W137,W138,W139,W140,W141,W142,W143,W144,W145,W146,W147,W148,W149,W150,W151,W152,W153,W154,W155,W156,W157,W158,W159,W160,W161,W162,W163,W164,W165,W166,W167,W168,W169,W170,W171,W172,W173,W174,W175,W176,W177,W178,W179,W180,W181,W182,W183,W184,W185,W186,W187,W188,W189,W190,W191,W192,W193,W194,W195,W196,W197,W198,W199,W200
e4,2541,3820,5896,7377,6698,4708,7284,4553,4437,5858,4313,2591,998,236,5202,5800,6168,6390,462,5710,3161,3905,6316,6863,2225,7029,6114,9626,8475,6455,7559,3984,8735,5559,4698,6287,8456,1475,8555,4229,6426,6301,7860,7171,1229,426,1693,9866,3118,8026,6761,123,3703,2999,6167,8804,5064,2712,8008,5452,8839,6842,8092,1439,5287,3546,3447,3610,5471,3018,1356,1925,2977,1519,3907,102,3088,5202,1919,5392,5651,6739,1221,8912,342,115,6444,8062,484,1335,1372,9063,6906,8246,947,5249,5025,7444,2386,7664,3700,2759,2318,3533,9154,8792,8015,603,5936,3060,8485,529,8896,5418,8266,7818,4452,6968,5257,4218,6240,9810,5417,9909,4777,1743,3569,6808,6619,3347,6001,1695,7908,2019,5130,9823,5962,4628,5218,1602,7955,9088,5073,2830,3325,6438,6743,8778,890,5486,2687,248,3123,3185,9041,4518,1031,238,9139,9242,2856,477,7494,3817,2730,2695,8072,4579,3969,1847,5882,4536,7964,3248,1911,9026,2114,6763,7993,6680,3801,6317,5640,6473,2879,3830,2610,4176,707,1702,4876,4779,4281,6561,4083,3172,2631,8157,5270,4860
Nf3,7313,994,3398,1436,175,7719,5946,6643,5212,3054,1954,3970,2971,8273,1764,6076,5455,3088,9182,222,3381,6717,3412,5154,226,7566,6311,7173,7673,9061,9312,7375,4968,3381,7284,3498,1679,5640,6159,636,3137,382,7049,460,6209,1956,8186,1505,1368,7686,7956,2832,5430,7164,8908,7833,910,5630,731,2321,4061,2753,7059,455,4919,3725,9068,7970,5699,6707,3327,5658,3930,718,7019,5290,161,5232,5155,8653,5730,2297,5616,3711,4366,6309,2882,1046,1231,3429,406,9013,9075,8668,9016,2319,9507,8008,4147,7353,4455,8491,1317,2025,7675,1079,8853,130,2818,1621,8578,2526,4668,420,4963,1628,3727,7409,8922,1104,6536,8096,7798,4525,6640,4527,8791,9025,2838,5883,2543,4489,5352,247,613,7043,7456,297,4843,7454,3895,1893,7212,5425,8112,3615,7438,2974,369,7172,974,8036,6626,2326,495,3296,172,1280,6046,5761,4490,9120,1442,8623,8406,2689,8883,9068,5728,6079,6261,6997,9872,5981,4881,745,8569,3777,9658,2334,4301,491,7603,7945,3539,6290,3758,7407,4578,1073,240,3740,1871,6809,3132,7141,2987,7581,2172,7282
d4,4359,8317,2310,1811,2051,4237,2174,3725,284,325,1413,6479,1385,1998,8331,1892,5923,157,2639,2713,5810,9561,6817,4054,1487,1454,1522,8664,3800,486,883,8144,3641,1895,5755,7021,8324,7301,2575,2359,2077,8824,5429,3145,4731,4246,3722,8849,3537,9008,5542,2651,3468,4964,8044,7050,8725,3165,8360,9566,1107,8605,5962,2348,6535,4404,6822,436,7325,1034,1836,4298,6584,3290,8526,9229,6910,3381,3075,1426,2144,9421,7095,6927,8384,1304,3315,9316,6549,8705,433,2371,4297,4924,9090,7655,3496,6282,2937,1985,4219,9213,7516,5449,8403,9540,7327,6103,1421,9160,5252,5543,6876,9443,2600,2637,9877,4165,4898,4306,5610,8209,4189,133,9258,6612,6676,8881,513,9401,3803,3845,5437,2222,4777,4054,5301,4971,1008,1780,7667,3014,3998,204,5060,7526,950,7772,7129,3456,6178,1354,8366,5509,2144,6404,7210,2131,2596,5043,4312,2326,9204,5053,1064,4685,6345,9521,5847,7163,6120,8476,4779,6137,7972,4553,6466,2309,2430,3608,1355,2824,3604,2925,5884,2695,3862,7285,5613,728,3552,6749,6327,5557,8948,890,6127,826,5980,3297
Nxd4,896,1467,5688,4100,2251,7112,8256,834,1777,4283,280,4114,7968,4315,2640,8452,3105,1306,3528,6849,5684,982,8980,8797,4091,6305,3672,3249,3695,7459,65

In [9]:
b_agent = Agent.Agent('B', chess_data)

In [10]:
b_agent.Q_table

,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,B15,B16,B17,B18,B19,B20,B21,B22,B23,B24,B25,B26,B27,B28,B29,B30,B31,B32,B33,B34,B35,B36,B37,B38,B39,B40,B41,B42,B43,B44,B45,B46,B47,B48,B49,B50,B51,B52,B53,B54,B55,B56,B57,B58,B59,B60,B61,B62,B63,B64,B65,B66,B67,B68,B69,B70,B71,B72,B73,B74,B75,B76,B77,B78,B79,B80,B81,B82,B83,B84,B85,B86,B87,B88,B89,B90,B91,B92,B93,B94,B95,B96,B97,B98,B99,B100,B101,B102,B103,B104,B105,B106,B107,B108,B109,B110,B111,B112,B113,B114,B115,B116,B117,B118,B119,B120,B121,B122,B123,B124,B125,B126,B127,B128,B129,B130,B131,B132,B133,B134,B135,B136,B137,B138,B139,B140,B141,B142,B143,B144,B145,B146,B147,B148,B149,B150,B151,B152,B153,B154,B155,B156,B157,B158,B159,B160,B161,B162,B163,B164,B165,B166,B167,B168,B169,B170,B171,B172,B173,B174,B175,B176,B177,B178,B179,B180,B181,B182,B183,B184,B185,B186,B187,B188,B189,B190,B191,B192,B193,B194,B195,B196,B197,B198,B199,B200
c5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
d6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
cxd4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Nf6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
a6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Nfxh5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [16]:
w_agent.chess_data

,Num Moves,W1,B1,W2,B2,W3,B3,W4,B4,W5,B5,W6,B6,W7,B7,W8,B8,W9,B9,W10,B10,W11,B11,W12,B12,W13,B13,W14,B14,W15,B15,W16,B16,W17,B17,W18,B18,W19,B19,W20,B20,W21,B21,W22,B22,W23,B23,W24,B24,W25,B25,W26,B26,W27,B27,W28,B28,W29,B29,W30,B30,W31,B31,W32,B32,W33,B33,W34,B34,W35,B35,W36,B36,W37,B37,W38,B38,W39,B39,W40,B40,W41,B41,W42,B42,W43,B43,W44,B44,W45,B45,W46,B46,W47,B47,W48,B48,W49,B49,W50,B50
Game 1,49,e4,c5,Nf3,d6,d4,cxd4,Nxd4,Nf6,Nc3,a6,Be3,e6,f3,b5,g4,h6,Qd2,Nbd7,O-O-O,Bb7,h4,b4,Na4,d5,Bh3,g5,Bg2,gxh4,Rxh4,dxe4,g5,Nd5,Rxe4,hxg5,Bxg5,Qa5,f4,Rh2,Nxe6,fxe6,Rxe6+,Kf7,Qd3,Bg7,Qf5+,Kg8,Rxd5,Qxa4,Re7,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Game 2,38,e4,e6,d4,d5,Nc3,Nf6,Bg5,Bb4,e5,h6,Be3,Ne4,Qg4,Kf8,a3,Bxc3+,bxc3,c5,Bd3,h5,Qf4,Qa5,Ne2,Nxc3,O-O,Nxe2+,Bxe2,Nc6,c4,cxd4,Bxd4,Nxd4,Qxd4,Bd7,cxd5,exd5,Bf3,Bc6,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Game 3,45,e4,c5,Nf3,Nc6,Bb5,d6,c3,Nf6,Qe2,e5,d4,exd4,cxd4,a6,e5,Nd5,exd6+,Be6,Bxc6+,bxc6,dxc5,Qa5+,Bd2,Qxc5,O-O,Kd7,Na3,Bxd6,Rab1,Bf5,Rbc1,Rhe8,Qd1,Qa7,Nc4,Bc7,Bc3,f6,Nd4,Be6,Nxc6,Kxc6,Bd4,Qb8,Qa4+,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Game 4,29,d4,Nf6,Bg5,d5,e3,g6,Bxf6,exf6,c4,dxc4,Bxc4,Bd6,Nc3,O-O,Nf3,Nd7,O-O,f5,Rc1,c6,Qc2,Nf6,Rfd1,Qe7,g3,Be6,Bxe6,Qxe6,a3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Game 5,42,e4,e5,Nf3,Nf6,Nxe5,d6,Nf3,Nxe4,d4,d5,Bd3,Nc6,O-O,Be7,c4,Nb4,Be2,O-O,Nc3,b6,a3,Nxc3,bxc3,Nc6,cxd5,Qxd5,Re1,Bb7,Bd3,Rae8,Qc2,h6,Bh7+,Kh8,Be4,Qd8,Bb2,Bf6,c4,Na5,Bxb7,Nxb7,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Game 104348,33,d4,d5,c4,e6,cxd5,exd5,Nc3,c6,Nf3,Bf5,Bg5,Be7,Bxe7,Qxe7,e3,Nd7,Bd3,Bxd3,Qxd3,Ngf6,O-O,Ne4,Rab1,a5,Qc2,f5,a3,O-O,b4,axb4,axb4,Ra3,Rb3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Game 104349,30,e4,c5,Nc3,g6,Nf3,Bg7,d4,cxd4,Nxd4,Nc6,Be3,d6,Qd2,Bd7,f3,Rc8,g4,Ne5,g5,Qa5,O-O-O,Nxf3,Nxf3,Bxc3,bxc3,Qa3+,Kb1,Rxc3,Bd4,Be6,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Game 104350,33,e4,g6,d4,Bg7,Nc3,Nh6,Bc4,O-O,Bb3,Kh8,h4,c5,Be3,Nc6,dxc5,f5,Qd2,Ng4,h5,Nxe3,Qxe3,gxh5,Rxh5,d5,Qh3,f4,Rxh7+,Kg8,Qh5,Qe8,Nxd5,Qxh5,Nf6#,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Game 104351,33,e4,d5,exd5,Qxd5,Nc3,Qa5,d4,Nf6,Nf3,c6,Bd2,Bg4,Be2,e6,O-O,Nbd7,h3,Bh5,a3,Qc7,Re1,Be7,Ne5,Bxe2,Qxe2,O-O,Rad1,Nxe5,dxe5,Nd5,Qg4,Nxc3,Bxc3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [19]:
import numpy as np

# Assuming q_table is your existing DataFrame
w_agent.Q_table = w_agent.Q_table.applymap(lambda x: np.random.randint(50, 10001))
w_agent.Q_table


,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,W11,W12,W13,W14,W15,W16,W17,W18,W19,W20,W21,W22,W23,W24,W25,W26,W27,W28,W29,W30,W31,W32,W33,W34,W35,W36,W37,W38,W39,W40,W41,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51,W52,W53,W54,W55,W56,W57,W58,W59,W60,W61,W62,W63,W64,W65,W66,W67,W68,W69,W70,W71,W72,W73,W74,W75,W76,W77,W78,W79,W80,W81,W82,W83,W84,W85,W86,W87,W88,W89,W90,W91,W92,W93,W94,W95,W96,W97,W98,W99,W100,W101,W102,W103,W104,W105,W106,W107,W108,W109,W110,W111,W112,W113,W114,W115,W116,W117,W118,W119,W120,W121,W122,W123,W124,W125,W126,W127,W128,W129,W130,W131,W132,W133,W134,W135,W136,W137,W138,W139,W140,W141,W142,W143,W144,W145,W146,W147,W148,W149,W150,W151,W152,W153,W154,W155,W156,W157,W158,W159,W160,W161,W162,W163,W164,W165,W166,W167,W168,W169,W170,W171,W172,W173,W174,W175,W176,W177,W178,W179,W180,W181,W182,W183,W184,W185,W186,W187,W188,W189,W190,W191,W192,W193,W194,W195,W196,W197,W198,W199,W200
e4,2541,3820,5896,7377,6698,4708,7284,4553,4437,5858,4313,2591,998,236,5202,5800,6168,6390,462,5710,3161,3905,6316,6863,2225,7029,6114,9626,8475,6455,7559,3984,8735,5559,4698,6287,8456,1475,8555,4229,6426,6301,7860,7171,1229,426,1693,9866,3118,8026,6761,123,3703,2999,6167,8804,5064,2712,8008,5452,8839,6842,8092,1439,5287,3546,3447,3610,5471,3018,1356,1925,2977,1519,3907,102,3088,5202,1919,5392,5651,6739,1221,8912,342,115,6444,8062,484,1335,1372,9063,6906,8246,947,5249,5025,7444,2386,7664,3700,2759,2318,3533,9154,8792,8015,603,5936,3060,8485,529,8896,5418,8266,7818,4452,6968,5257,4218,6240,9810,5417,9909,4777,1743,3569,6808,6619,3347,6001,1695,7908,2019,5130,9823,5962,4628,5218,1602,7955,9088,5073,2830,3325,6438,6743,8778,890,5486,2687,248,3123,3185,9041,4518,1031,238,9139,9242,2856,477,7494,3817,2730,2695,8072,4579,3969,1847,5882,4536,7964,3248,1911,9026,2114,6763,7993,6680,3801,6317,5640,6473,2879,3830,2610,4176,707,1702,4876,4779,4281,6561,4083,3172,2631,8157,5270,4860
Nf3,7313,994,3398,1436,175,7719,5946,6643,5212,3054,1954,3970,2971,8273,1764,6076,5455,3088,9182,222,3381,6717,3412,5154,226,7566,6311,7173,7673,9061,9312,7375,4968,3381,7284,3498,1679,5640,6159,636,3137,382,7049,460,6209,1956,8186,1505,1368,7686,7956,2832,5430,7164,8908,7833,910,5630,731,2321,4061,2753,7059,455,4919,3725,9068,7970,5699,6707,3327,5658,3930,718,7019,5290,161,5232,5155,8653,5730,2297,5616,3711,4366,6309,2882,1046,1231,3429,406,9013,9075,8668,9016,2319,9507,8008,4147,7353,4455,8491,1317,2025,7675,1079,8853,130,2818,1621,8578,2526,4668,420,4963,1628,3727,7409,8922,1104,6536,8096,7798,4525,6640,4527,8791,9025,2838,5883,2543,4489,5352,247,613,7043,7456,297,4843,7454,3895,1893,7212,5425,8112,3615,7438,2974,369,7172,974,8036,6626,2326,495,3296,172,1280,6046,5761,4490,9120,1442,8623,8406,2689,8883,9068,5728,6079,6261,6997,9872,5981,4881,745,8569,3777,9658,2334,4301,491,7603,7945,3539,6290,3758,7407,4578,1073,240,3740,1871,6809,3132,7141,2987,7581,2172,7282
d4,4359,8317,2310,1811,2051,4237,2174,3725,284,325,1413,6479,1385,1998,8331,1892,5923,157,2639,2713,5810,9561,6817,4054,1487,1454,1522,8664,3800,486,883,8144,3641,1895,5755,7021,8324,7301,2575,2359,2077,8824,5429,3145,4731,4246,3722,8849,3537,9008,5542,2651,3468,4964,8044,7050,8725,3165,8360,9566,1107,8605,5962,2348,6535,4404,6822,436,7325,1034,1836,4298,6584,3290,8526,9229,6910,3381,3075,1426,2144,9421,7095,6927,8384,1304,3315,9316,6549,8705,433,2371,4297,4924,9090,7655,3496,6282,2937,1985,4219,9213,7516,5449,8403,9540,7327,6103,1421,9160,5252,5543,6876,9443,2600,2637,9877,4165,4898,4306,5610,8209,4189,133,9258,6612,6676,8881,513,9401,3803,3845,5437,2222,4777,4054,5301,4971,1008,1780,7667,3014,3998,204,5060,7526,950,7772,7129,3456,6178,1354,8366,5509,2144,6404,7210,2131,2596,5043,4312,2326,9204,5053,1064,4685,6345,9521,5847,7163,6120,8476,4779,6137,7972,4553,6466,2309,2430,3608,1355,2824,3604,2925,5884,2695,3862,7285,5613,728,3552,6749,6327,5557,8948,890,6127,826,5980,3297
Nxd4,896,1467,5688,4100,2251,7112,8256,834,1777,4283,280,4114,7968,4315,2640,8452,3105,1306,3528,6849,5684,982,8980,8797,4091,6305,3672,3249,3695,7459,65

In [43]:
import random

In [46]:
def choose_action(self, environ_state: dict[str, str, list[str]], curr_game: str = 'Game 1') -> str:
    """Chooses the next chess move for the agent based on the current state.
    Args:
        environ_state (dict): A dictionary containing the current state.
        curr_game (str): A string indicating the current game being played. 
    Returns:
        str: A string representing the chess move chosen by the agent.
    """
    # check if any of the legal moves is not already in the Q table
    moves_not_in_Q_table: list[str] = [move for move in environ_state['legal_moves'] if move not in self.Q_table.index]

    if moves_not_in_Q_table:
        self.update_Q_table(moves_not_in_Q_table)

    if self.is_trained:
        return self.policy_game_mode(environ_state['legal_moves'])
    else:
        return self.policy_training_mode(curr_game, environ_state["curr_turn"])

def policy_training_mode(self, curr_game: str, curr_turn: str) -> str:
    """Determines how the agents choose a move at each turn during training.
    Args:
        curr_game: A string representing the current game being played.
        curr_turn: A string representing the current turn, e.g. 'W1'.
    Returns:
        str: A string representing the chess move chosen by the agent.
    """
    return self.chess_data.at[curr_game, curr_turn]
### end of policy_training_mode ###


def policy_game_mode(self, legal_moves: list[str]) -> str:
    """Determines how the agent chooses a move during a game between a human player and the agent.
    The agent searches its Q table to find the moves with the highest Q values at each turn. 
    sometimes the agent will pick a random move. 

    Args:
        legal_moves: A list of strings representing the legal moves for the current turn.
    Returns:
        str: A string representing the chess move chosen by the agent.
    """
    dice_roll = get_number_with_probability(0.1)
    legal_moves_in_q_table = self.Q_table[curr_turn].loc[self.Q_table[curr_turn].index.intersection(legal_moves)]

    if dice_roll == 1:
        chess_move = legal_moves_in_q_table.sample().index[0]
    else:
        chess_move = legal_moves_in_q_table.idxmax()
    return chess_move
### end of policy_game_mode ###

def get_number_with_probability(probability: float) -> int:
    """Generate a random number with a given probability.
    Args:
        probability (float): A float representing the probability of generating a 1.
    Returns:
        int: A random integer value of either 0 or 1.
    """
    if random.random() < probability:
        return 1
    else:
        return 0

In [30]:
environ_state = {'turn_index': 0, 'curr_turn': 'W2', 'legal_moves': [
    "e3", "e4", "d3", "d4", "c3", "c4", "f3", "f4", "g3", "g4", "h3", "h4", 
    "a3", "a4", "b3", "b4", "b1c3", "b1a3", "g1f3", "g1h3"
]}

In [35]:
w_agent.is_trained = True

In [47]:
chess_move_1 = w_agent.choose_action(environ_state)

NameError: name 'random' is not defined

In [34]:
chess_move_1

'Nf3'